#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [74]:
import pandas as pd
import pymongo
from pymongo import MongoClient

In [75]:
client = MongoClient('mongodb://mongo:27017/')
database = client['exercices']
collection = database['kickstarter']

In [76]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [77]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [78]:
df_ks = df_ks.dropna()

### Importer les données

In [79]:
db = client.df
print(type(db))

<class 'pymongo.database.Database'>


In [83]:
#collection = db['kickstarter']
collection.delete_many({})
collection.insert_many(df_ks.to_dict('records'))

In [84]:
collection.find_one()

{'_id': ObjectId('5fbbbb60cb2bdb84d9c607c5'),
 'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': '2011-08-17 06:31:31',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [86]:
cur = collection.find().sort([("usd_pledged_real", -1)]).limit(5)
list(cur)

[{'_id': ObjectId('5fbbbb61cb2bdb84d9c72fe3'),
  'ID': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': '50000.0',
  'launched': '2014-07-08 10:14:37',
  'pledged': '13285226.36',
  'state': 'successful',
  'backers': '62642',
  'country': 'US',
  'usd pledged': '13285226.36',
  'usd_pledged_real': 13285226.36},
 {'_id': ObjectId('5fbbbb61cb2bdb84d9c78a0d'),
  'ID': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': '1000000.0',
  'launched': '2016-05-24 15:49:52',
  'pledged': '12779843.49',
  'state': 'successful',
  'backers': '66673',
  'country': 'US',
  'usd pledged': '12779843.49',
  'usd_pledged_real': 12779843.49},
 {'_id': ObjectId('5fbbbb61cb2bdb84d9c7bea4'),
  'ID': 1033978702,
  'name': 'OUY

## Question 2

In [91]:
cur = collection.find({"state":"successful"})
next(cur)
print(len(list(cur)))

52997


## Question 3

In [99]:
cur = collection.aggregate([{"$group" : {"_id" : "$category", "sum" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Plays', 'sum': 539},
 {'_id': 'Classical Music', 'sum': 1064},
 {'_id': 'Accessories', 'sum': 1203},
 {'_id': 'Product Design', 'sum': 8885},
 {'_id': 'Hardware', 'sum': 1430},
 {'_id': 'Apps', 'sum': 2535},
 {'_id': 'Fashion', 'sum': 3379},
 {'_id': 'Experimental', 'sum': 357},
 {'_id': 'Indie Rock', 'sum': 2192},
 {'_id': 'Video', 'sum': 147},
 {'_id': 'Events', 'sum': 321},
 {'_id': 'Webcomics', 'sum': 259},
 {'_id': 'Literary Journals', 'sum': 118},
 {'_id': 'Art Books', 'sum': 1065},
 {'_id': 'Faith', 'sum': 439},
 {'_id': 'Musical', 'sum': 367},
 {'_id': 'Apparel', 'sum': 2827},
 {'_id': 'People', 'sum': 440},
 {'_id': 'Pop', 'sum': 1302},
 {'_id': 'Art', 'sum': 3358},
 {'_id': 'Music', 'sum': 5293},
 {'_id': 'Ready-to-wear', 'sum': 332},
 {'_id': 'Drama', 'sum': 871},
 {'_id': 'R&B', 'sum': 172},
 {'_id': 'Candles', 'sum': 168},
 {'_id': 'Romance', 'sum': 74},
 {'_id': 'Young Adult', 'sum': 328},
 {'_id': 'Nonfiction', 'sum': 3390},
 {'_id': 'Web', 'sum': 2017},
 {'_id

## Question 4

In [106]:
cur = collection.find({"$and":[{"launched":{"$lt":"2016-01-01 00:00:00"}}, {"country":"FR"}]})
print(len(list(cur)))

330


## Question 5

In [112]:
cur = collection.find({"$and":[{"goal":{"$gt":200000}}, {"country":"US"}]})
next(cur)
print(len(list(cur)))

1025


## Question 6 

In [115]:
collection.create_index([("name",  "text")])
cur = collection.find( { "$text": { "$search": "Sport" } } )
print(len(list(cur)))

316
